
1. Обучить любую модель классификации на датасете IRIS до применения PCA и после него. Сравнить качество классификации по отложенной выборке.


In [93]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets

In [94]:
X, y = datasets.load_iris(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [95]:
def e_metrics(x1, x2):
    
    distance = 0
    for i in range(len(x1)):
        distance += np.square(x1[i] - x2[i])
    
    return np.sqrt(distance)

In [96]:
def knn(x_train, y_train, x_test, k):
    
    answers = []
    for x in x_test:
        test_distances = []
            
        for i in range(len(x_train)):
            
            distance = e_metrics(x, x_train[i])
            
            test_distances.append((distance, y_train[i]))
        
        classes = {class_item: 0 for class_item in set(y_train)}
        
        for d in sorted(test_distances)[0:k]:
            if d[0] == 0:
                classes[d[1]] += 1
                continue    
            
            classes[d[1]] += 1/d[0]

        answers.append(sorted(classes, key=classes.get)[-1])

    return answers

def accuracy(pred, y):
    return (sum(pred == y) / len(y))

In [97]:
k = 5
for k in range(1, 20):
    y_pred = knn(X_train, y_train, X_test, k)

    print(f'Точность алгоритма при k = {k}: {accuracy(y_pred, y_test):.3f}')

Точность алгоритма при k = 1: 1.000
Точность алгоритма при k = 2: 1.000
Точность алгоритма при k = 3: 1.000
Точность алгоритма при k = 4: 1.000
Точность алгоритма при k = 5: 1.000
Точность алгоритма при k = 6: 1.000
Точность алгоритма при k = 7: 0.967
Точность алгоритма при k = 8: 1.000
Точность алгоритма при k = 9: 0.967
Точность алгоритма при k = 10: 0.967
Точность алгоритма при k = 11: 0.967
Точность алгоритма при k = 12: 0.967
Точность алгоритма при k = 13: 0.967
Точность алгоритма при k = 14: 0.967
Точность алгоритма при k = 15: 0.967
Точность алгоритма при k = 16: 0.967
Точность алгоритма при k = 17: 0.967
Точность алгоритма при k = 18: 0.967
Точность алгоритма при k = 19: 0.967


In [98]:
# Для начала отмасштабируем выборку
X_ = X.astype(float)

rows, cols = X_.shape

# центрирование - вычитание из каждого значения среднего по строке
means = X_.mean(0)
for i in range(rows):
    for j in range(cols):
        X_[i, j] -= means[j]

# деление каждого значения на стандартное отклонение
std = np.std(X_, axis=0)
for i in range(cols):
    for j in range(rows):
        X_[j][i] /= std[i]

In [99]:
# Найдем собственные векторы и собственные значения
 
covariance_matrix = X_.T.dot(X_)

eig_values, eig_vectors = np.linalg.eig(covariance_matrix)

# сформируем список кортежей (собственное значение, собственный вектор)
eig_pairs = [(np.abs(eig_values[i]), eig_vectors[:, i]) for i in range(len(eig_values))]

# и отсортируем список по убыванию собственных значений
eig_pairs.sort(key=lambda x: x[0], reverse=True)

print('Собственные значения в порядке убывания:')
for i in eig_pairs:
    print(i[0])

Собственные значения в порядке убывания:
437.77467247979905
137.1045707202107
22.013531335697174
3.1072254642928927


In [100]:
eig_sum = sum(eig_values)
var_exp = [(i / eig_sum) * 100 for i in sorted(eig_values, reverse=True)]
cum_var_exp = np.cumsum(var_exp)
print(f'Доля дисперсии, описвыаемая каждой из компонент \n{var_exp}')

# а теперя оценим кумулятивную (то есть накапливаемую) дисперсию при учитывании каждой из компонент
print(f'Кумулятивная доля дисперсии по компонентам \n{cum_var_exp}')

Доля дисперсии, описвыаемая каждой из компонент 
[72.96244541329986, 22.85076178670179, 3.6689218892828626, 0.5178709107154822]
Кумулятивная доля дисперсии по компонентам 
[ 72.96244541  95.8132072   99.48212909 100.        ]


In [101]:
var_exp

[72.96244541329986, 22.85076178670179, 3.6689218892828626, 0.5178709107154822]

In [102]:
# Сформируем вектор весов из собственных векторов, соответствующих первым двум главным компонентам
W = np.hstack((eig_pairs[0][1].reshape(4,1), eig_pairs[1][1].reshape(4,1)))

print(f'Матрица весов W:\n', W)

Матрица весов W:
 [[ 0.52106591 -0.37741762]
 [-0.26934744 -0.92329566]
 [ 0.5804131  -0.02449161]
 [ 0.56485654 -0.06694199]]


In [103]:
X_train, X_test, y_train, y_test = train_test_split(X.dot(W), y, test_size=0.2, random_state=1)

In [104]:

for k in range(1, 20):
    y_pred = knn(X_train, y_train, X_test, k)

    print(f'Точность алгоритма при k = {k}: {accuracy(y_pred, y_test):.3f}')

Точность алгоритма при k = 1: 0.967
Точность алгоритма при k = 2: 0.967
Точность алгоритма при k = 3: 0.967
Точность алгоритма при k = 4: 0.967
Точность алгоритма при k = 5: 0.967
Точность алгоритма при k = 6: 0.967
Точность алгоритма при k = 7: 0.967
Точность алгоритма при k = 8: 0.967
Точность алгоритма при k = 9: 0.967
Точность алгоритма при k = 10: 0.967
Точность алгоритма при k = 11: 0.967
Точность алгоритма при k = 12: 0.967
Точность алгоритма при k = 13: 0.967
Точность алгоритма при k = 14: 0.967
Точность алгоритма при k = 15: 0.967
Точность алгоритма при k = 16: 0.967
Точность алгоритма при k = 17: 0.967
Точность алгоритма при k = 18: 0.967
Точность алгоритма при k = 19: 0.967
